In [1]:
import nbimporter
import pandas as pd
import math
import time
#import ace_tools as tools

In [2]:
from graph_utilities import lire_graphe, afficher_graphe, visualiser_graphe_par_etape
from graph_utilities import generate_cyclic_graph, generate_chain_graph, generate_spider_graph
from mtd_math import solver, solver_beta, solver_beta_dichotomic
from algo_recherche import recherche_profondeur, recherche_largeur, couverture_balle

In [3]:
# 10 sommets (3.16)
#fichier = r'instances\Stranke94\Stranke94.mtx'

# 29 sommets (5.38)
#fichier = r'instances\bn-mouse_visual-cortex_1\bn-mouse_visual-cortex_1.mtx'

# 34 sommets (5,83)
#fichier = r'instances\karate\karate.mtx'

# 62 sommets (7.87)
#fichier = r'instances\dolphins\dolphins.mtx'

# 105 sommets (12.88)
#fichier = r'instances\polbooks\polbooks.mtx'

# 258 sommets (16.06)
#fichier = r'instances\sphere3\sphere3.mtx'

# 379 sommets (19.47)
#fichier = r'instances\ca-netscience\ca-netscience.mtx'

# 7057 sommets (84.005)
#fichier = r'instances\fb-pages-government\fb-pages-government.mtx'

# 11631 sommets (107.84)
#fichier = r'instances\web-wiki-crocodile\web-wiki-crocodile.mtx'

# 196 591 sommets (443.38)
#fichier = r'instances\loc-gowalla_edges\loc-gowalla_edges.mtx'

#graphe = lire_graphe(fichier)

In [4]:
# Graphe cyclique (4)
#graphe = generate_cyclic_graph(16)

# Graphe en chaîne (4)
#graphe = generate_chain_graph(16)

# Graphe en étoile/spider (4.58)
#graphe = generate_spider_graph(0, leg_length=4, num_legs=5)

In [5]:
#afficher_graphe(graphe)

---

In [6]:
# Définition des graphes à tester
graphes_excep = {
    "Cycle_7": generate_cyclic_graph(7),
    "Chaine_7": generate_chain_graph(7),
    "Etoile_7": generate_spider_graph(0, leg_length=3, num_legs=2),
    "Cycle_16": generate_cyclic_graph(16),
    "Chaine_16": generate_chain_graph(16),
    "Etoile_16": generate_spider_graph(0, leg_length=5, num_legs=3),
    "Cycle_18": generate_cyclic_graph(18),
    "Chaine_18": generate_chain_graph(18),
    "Etoile_21": generate_spider_graph(0, leg_length=4, num_legs=5),
}

In [7]:
# Définition des graphes à tester
graphes = {
    "Stranke94": lire_graphe(r'instances/Stranke94/Stranke94.mtx'),
    "mouse_visual-cortex_1": lire_graphe(r'instances/bn-mouse_visual-cortex_1/bn-mouse_visual-cortex_1.mtx'),
    "karate": lire_graphe(r'instances/karate/karate.mtx'),
    #"dolphins": lire_graphe(r'instances/dolphins/dolphins.mtx'),
    #"polbooks": lire_graphe(r'instances/polbooks/polbooks.mtx'),
    #"sphere3": lire_graphe(r'instances/sphere3/sphere3.mtx'),
    #"ca-netscience": lire_graphe(r'instances/ca-netscience/ca-netscience.mtx')
}

In [8]:
def df_resultats(graphes):
    # Initialisation du tableau des résultats
    resultats = []

    # Parcours de chaque graphe
    for nom, graphe in graphes.items():
        nb_sommets = len(graphe)
        #nb_aretes = sum(len(v) for v in graphe.values()) // 2  # Division par 2 car graphe non orienté
        k = nb_sommets ** 0.5

        #try:
        #    start = time.perf_counter()
        #    solver_result, solver_cost = solver(graphe)
        #    solver_time = time.perf_counter() - start
        #except Exception as e:
        #    solver_result, solver_cost, solver_time = str(e), None, None

        try:
            start = time.perf_counter()
            solver_beta_result, solver_beta_cost  = solver_beta(graphe)
            solver_beta_time = time.perf_counter() - start
        except Exception as e:
            solver_beta_result, solver_beta_cost, solver_beta_time = str(e), None, None

        try:
            start = time.perf_counter()
            solver_beta_dico_result, solver_beta_dico_cost = solver_beta_dichotomic(graphe)
            solver_beta_dico_time = time.perf_counter() - start
        except Exception as e:
            solver_beta_dico_result, solver_beta_dico_cost, solver_beta_dico_time = str(e), None, None

        try:
            start = time.perf_counter()
            recherche_p_res, recherche_p_cout = recherche_profondeur(graphe)
            recherche_p_time = time.perf_counter() - start
        except Exception as e:
            recherche_p_res, recherche_p_cout, recherche_p_time = str(e), None, None
        
        try:
            start = time.perf_counter()
            etat, recherche_l_res, recherche_l_cout = recherche_largeur(graphe)
            recherche_l_time = time.perf_counter() - start
        except Exception as e:
            recherche_l_res, recherche_l_cout, recherche_l_time = str(e), None, None

        try:
            start = time.perf_counter()
            couverture_balle_res, couverture_balle_cout = couverture_balle(graphe)
            couverture_balle_time = time.perf_counter() - start
        except Exception as e:
            couverture_balle_res, couverture_balle_cout, couverture_balle_time  = str(e), None, None
        
        # Ajout des résultats au tableau
        resultats.append({
            "Graphe": nom,
            "V": nb_sommets,
            "sqrt(V)": k,
            "B_max": math.ceil(k),

            #"Solveur": solver_result,
            #"Cout Solveur": solver_cost,
            #"temps Solveur": solver_time,

            "Solveur_beta": solver_beta_result,
            "Cout Solveur_beta": solver_beta_cost,
            "temps Solveur_beta": solver_beta_time,

            "Solveur dico": solver_beta_dico_result,
            "Cout Solveur dico": solver_beta_dico_cost,
            "temps Solveur dico": solver_beta_dico_time,

            "Recherche Profoncdeur": recherche_p_res,
            "Cout RP": recherche_p_cout,
            "temps RP": recherche_p_time,

            "Recherche Largeur": recherche_l_res,
            "Cout RL": recherche_l_cout,
            "temps RL": recherche_l_time,
            
            "Couverture des balles": couverture_balle_res,
            "Cout CB": couverture_balle_cout,
            "temps CB": couverture_balle_time
        })


    # Convertir en DataFrame et afficher
    df = pd.DataFrame(resultats)

    return df

In [9]:
res_graphes_excep = df_resultats(graphes_excep)
res_graphes_excep

Set parameter Username
Set parameter LicenseID to value 2601900
Academic license - for non-commercial use only - expires 2025-12-18


,Graphe,V,sqrt(V),B_max,Solveur_beta,Cout Solveur_beta,temps Solveur_beta,Solveur dico,Cout Solveur dico,temps Solveur dico,Recherche Profoncdeur,Cout RP,temps RP,Recherche Largeur,Cout RL,temps RL,Couverture des balles,Cout CB,temps CB
0,Cycle_7,7,2.645751,3,"[4, 0, 6]",3,0.104055,"[4, 0, 6]",3,0.007482,"[0, 2, 4]",3,0.000096,"[0, 2, 4]",3,0.000752,"[(2, 2), (6, 1)]",3,0.001871
1,Chaine_7,7,2.645751,3,"[3, 0, 6]",3,0.008533,"[3, 0, 6]",3,0.005173,"[1, 3, 5]",4,0.000061,"[0, 4, 6]",3,0.000793,"[(4, 2), (1, 1)]",3,0.001885
2,Etoile_7,7,2.645751,3,"[0, 2, 6]",3,0.008422,"[0, 2, 6]",3,0.005369,"[0, 2, 6]",3,0.000071,"[0, 2, 6]",3,0.000988,"[(4, 2), (2, 1)]",3,0.001304
3,Cycle_16,16,4.000000,4,"[15, 9, 5, 3]",4,0.034894,"[15, 9, 5, 3]",4,0.016543,"[0, 2, 4, 6, 8, 10]",6,0.000085,"[0, 6, 10, 12]",4,0.230150,"[(9, 3), (15, 2), (4, 1), (2, 0)]",4,0.284079
4,Chaine_16,16,4.000000,4,"[12, 2, 7, 5]",4,0.017899,"[12, 2, 7, 5]",4,0.016576,"[1, 3, 5, 7, 9, 11, 13, 15]",8,0.000101,"[3, 9, 13, 15]",4,0.475652,"[(8, 3), (2, 2), (14, 1), (12, 0)]",4,0.268438
5,Etoile_16,16,4.000000,4,"[11, 5, 9, 15]",4,0.019799,"[11, 5, 9, 15]",4,0.016865,"[0, 2, 4, 9, 15]",5,0.000125,"[1, 8, 14, 5]",4,0.272324,"[(11, 3), (3, 2), (9, 1), (15, 0)]",4,0.333135
6,Cycle_18,18,4.242641,5,"[14, 3, 8, 10, 11]",5,0.049025,"[14, 3, 8, 10, 11]",5,0.035857,"[0, 2, 4, 6, 8, 10]",7,0.000128,"[0, 1, 7, 11, 13]",5,2.810250,"[(6, 4), (14, 3), (1, 1)]",5,1.087103
7,Chaine_18,18,4.242641,5,"[14, 3, 8, 10, 11]",5,0.044485,"[14, 3, 8, 10, 11]",5,0.028703,"[1, 3, 5, 7, 9, 11, 13, 15, 17]",9,0.000100,"[0, 5, 11, 15, 17]",5,3.274407,"[(6, 4), (14, 3), (1, 1)]",5,1.357123
8,Etoile_21,21,4.582576,5,"[0, 19, 16, 18, 1]",5,0.036025,"[0, 19, 16, 18, 1]",5,0.023810,"[0, 2, 7, 12]",5,0.000063,"[0, 1, 2, 3, 4]",5,8.231312,"[(0, 4)]",5,4.061946


In [10]:
res_graphes = df_resultats(graphes)
res_graphes

,Graphe,V,sqrt(V),B_max,Solveur_beta,Cout Solveur_beta,temps Solveur_beta,Solveur dico,Cout Solveur dico,temps Solveur dico,Recherche Profoncdeur,Cout RP,temps RP,Recherche Largeur,Cout RL,temps RL,Couverture des balles,Cout CB,temps CB
0,Stranke94,10,3.162278,4,"[7, 5, 1]",3,0.012838,"[7, 5, 1]",3,0.006925,"[3, 6]",3,0.000047,"[3, 1, 6]",3,0.001717,"[(3, 2)]",3,0.007591
1,mouse_visual-cortex_1,29,5.385165,6,"[5, 10, 1, 24]",4,0.049155,"[5, 10, 1, 24]",4,0.059441,"[9, 2, 1, 6]",4,0.000096,"[1, 19, 22, 27]",4,3.777618,"[(28, 3), (10, 1)]",4,7.155019
2,karate,34,5.830952,6,"[20, 32, 17]",3,0.030538,"[20, 32, 17]",3,0.032331,"[34, 1, 17]",3,0.000175,"[1, 33, 27]",3,0.338840,"[(32, 2), (17, 0)]",3,2.182968


In [11]:
num = 2
df = res_graphes_excep
graphe = graphes_excep[df.iloc[num]["Graphe"]]

#visualiser_graphe_par_etape(graphe, [centre for centre, rayon in df.iloc[num]["Couverture des balles"]])
#visualiser_graphe_par_etape(graphe, df.iloc[num]["Recherche Largeur"])

## Comparaison des resultats

In [12]:
df_concat = pd.concat([res_graphes_excep, res_graphes], ignore_index=True)

### Comparaison des couts

In [14]:
df_cout = df_concat[["Graphe", "V", "sqrt(V)", "B_max", "Cout Solveur_beta", "Cout Solveur dico", "Cout RL", "Cout CB", "Cout RP"]]
df_cout

,Graphe,V,sqrt(V),B_max,Cout Solveur_beta,Cout Solveur dico,Cout RL,Cout CB,Cout RP
0,Cycle_7,7,2.645751,3,3,3,3,3,3
1,Chaine_7,7,2.645751,3,3,3,3,3,4
2,Etoile_7,7,2.645751,3,3,3,3,3,3
3,Cycle_16,16,4.000000,4,4,4,4,4,6
4,Chaine_16,16,4.000000,4,4,4,4,4,8
5,Etoile_16,16,4.000000,4,4,4,4,4,5
6,Cycle_18,18,4.242641,5,5,5,5,5,7
7,Chaine_18,18,4.242641,5,5,5,5,5,9
8,Etoile_21,21,4.582576,5,5,5,5,5,5
9,Stranke94,10,3.162278,4,3,3,3,3,3


### Comparaisons du temps d'execution

In [15]:
df_temps = df_concat[["Graphe", "V", "temps Solveur_beta", "temps Solveur dico", "temps RL", "temps CB", "temps RP"]]
df_temps

,Graphe,V,temps Solveur_beta,temps Solveur dico,temps RL,temps CB,temps RP
0,Cycle_7,7,0.104055,0.007482,0.000752,0.001871,0.000096
1,Chaine_7,7,0.008533,0.005173,0.000793,0.001885,0.000061
2,Etoile_7,7,0.008422,0.005369,0.000988,0.001304,0.000071
3,Cycle_16,16,0.034894,0.016543,0.230150,0.284079,0.000085
4,Chaine_16,16,0.017899,0.016576,0.475652,0.268438,0.000101
5,Etoile_16,16,0.019799,0.016865,0.272324,0.333135,0.000125
6,Cycle_18,18,0.049025,0.035857,2.810250,1.087103,0.000128
7,Chaine_18,18,0.044485,0.028703,3.274407,1.357123,0.000100
8,Etoile_21,21,0.036025,0.023810,8.231312,4.061946,0.000063
9,Stranke94,10,0.012838,0.006925,0.001717,0.007591,0.000047
